In [2]:
### Initialize & Read in data ###

from pylab import *
import numpy as np
import scipy as sp
import time
import os

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from matplotlib.collections import LineCollection
#%matplotlib inline 
# Normal matplotlib => just wrap in $ for latex

#import mpld3 # This is a widget for zooming in and manipulating plots
#mpld3.enable_notebook()

from sklearn import gaussian_process

In [3]:
### 27912 tracks numbers, 27891 total tracks ###
### Cut matrix: 1000 track numbersm 992 total tracks ###
### Create data matrix ###

dir_path = '/Users/michaelzevin/Documents/Stellar Evolution/Data/npy_resampled_cut/'

### Read in the inputs for each track ###
inputs = []
outputs_1 = []
outputs_2 = []

for file in os.listdir(dir_path):
    x = np.load(dir_path + file)
    
    inputs.append([[float(x['Mbh_init'])],[float(x['M2_init'])],[float(x['P_init'])]])
        
    outputs_1.append(x['xr1'])
    outputs_2.append(x['xr2'])
        
    print file
    # Could also try to run GP for each timestep separately
        
inputs = np.reshape(inputs,[len(inputs),3])

file1.npz
file10.npz
file100.npz
file1000.npz
file101.npz
file102.npz
file103.npz
file104.npz
file105.npz
file106.npz
file107.npz
file108.npz
file109.npz
file11.npz
file110.npz
file111.npz
file112.npz
file113.npz
file114.npz
file115.npz
file116.npz
file117.npz
file118.npz
file119.npz
file12.npz
file120.npz
file121.npz
file122.npz
file123.npz
file124.npz
file125.npz
file126.npz
file127.npz
file128.npz
file129.npz
file13.npz
file130.npz
file131.npz
file132.npz
file133.npz
file134.npz
file135.npz
file136.npz
file137.npz
file138.npz
file139.npz
file14.npz
file140.npz
file141.npz
file142.npz
file143.npz
file144.npz
file145.npz
file146.npz
file147.npz
file148.npz
file149.npz
file150.npz
file151.npz
file152.npz
file153.npz
file154.npz
file155.npz
file156.npz
file157.npz
file158.npz
file159.npz
file160.npz
file161.npz
file162.npz
file163.npz
file164.npz
file165.npz
file166.npz
file167.npz
file168.npz
file169.npz
file170.npz
file171.npz
file172.npz
file173.npz
file174.npz
file175.npz
file176.np

In [5]:
'''Run some time evolution tracks for both resampling'''

### Set up plots ###
fig=plt.figure(figsize = (12,9), facecolor = 'white')
fig.subplots_adjust(wspace=.5,hspace=.5)
plt.title('time evolution for different output parameters', size=20)

ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)
axs=[ax1,ax2,ax3,ax4]

#set up the line collections
lc1 = LineCollection([[(0, 0)]], linestyles = ['solid'], colors = ['blue'])
lc2 = LineCollection([[(0, 0)]], linestyles = ['dashed'], colors = ['red'])

#plt.xlabel('time (Gyr)', labelpad=20, size=15)
#plt.ylabel('logT', rotation=90, labelpad=20, size=15)

### POSSIBLE OUTPUTS TO PLOT ###
'''
(0'log_dt', 1'he_core_mass', 2'LH', 3'LHe', 4'Teff', 5'L', 
 6'R', 7'logg', 8'center_H1', 9'center_He4', 10'num_retries', 
 11'model', 12'age', 13'P', 14'A', 15'rl_relative_overflow', 16'M2', 
 17'Mbh', 18'log_abs_mdot', 19'xfer_fraction')
 '''
age_index = 12
timesteps = np.linspace(0,999,1000)

### Choose the track number you want to look at ###
track = 500
print 'The initial conditions for track %i are: Mbh = %f, M2 = %f, & P = %f' % (track,inputs[track,0],inputs[track,1],inputs[track,2])
    
### Choose 4 outputs to plot time evolution ###
evo=[4,5,16,17]    

### For labeling, reference a sample path ###
sample_path = '/Users/michaelzevin/Documents/Stellar Evolution/Data/npy_resampled/file1.npz'
x = np.load(sample_path)


for i in xrange(len(axs)):
    
    # Set up subplot
    ax = axs[i]
    ax = fig.add_subplot(2,2,i+1)
    ax.set_xlabel('time', rotation=0, labelpad=20, size=12)
    ax.set_ylabel(x['xr1'].dtype.names[evo[i]], rotation=90, labelpad=20, size=12)
    ax.legend([lc1, lc2], ['mass resampling', 'time resampling'], loc=0)
    
    # Pick out interpolation for each correlation and plot
    resample_1=[]
    resample_2=[]
    time_1=[]
    time_2=[]
    for j in xrange(1000):
        resample_1.append(outputs_1[track][j][evo[i]])
        resample_2.append(outputs_2[track][j][evo[i]])
        time_1.append((outputs_1[track][j][age_index])/10**9)
        time_2.append((outputs_2[track][j][age_index])/10**9)
    ax.plot(time_1, resample_1, 'b-')
    ax.plot(time_2, resample_2, 'r--')
    
plt.show()


The initial conditions for track 500 are: Mbh = 8.000000, M2 = 0.995086, & P = 1.494279


In [ ]:
type(outputs_1)

In [ ]:
'''Perform Leave-One-Out Validation for each of the resamplings'''
temp = list(inputs)
q=randint(0,len(inputs))
random_IC = array(temp.pop(q))
temp_out_1 = list(outputs_1)
temp_out_2 = list(outputs_2)
temp_out_1.pop(q)
temp_out_2.pop(q)

# temp are the inputs/outputs with the one value missing. Takes time to convert to array. 
temp_out_1 = array(temp_out_1)
temp_out_2 = array(temp_out_2)

def GP_Nd(x, y):
    gp = gaussian_process.GaussianProcess(corr='absolute_exponential', theta0=1e-2, thetaL=1e-4, thetaU=1e-1,nugget=2e-5)
    ''' other corr: 'absolute_exponential', 'squared_exponential', 'generalized_exponential', 'cubic', 'linear' '''
    x = np.atleast_2d(x)
    y = np.atleast_2d(y).T
    
    gp.fit(x, y)
    
    x_pred = np.atleast_2d(random_IC)
    y_pred, sigma2_pred = gp.predict(x_pred, eval_MSE=True)
    sigma = sqrt(sigma2_pred)
    
    return x_pred, y_pred, sigma

random_IC

In [ ]:
### Run GP for each timestep, not including the one left out ###
x_pred_1=[]; y_pred_1=[]; sigma_1=[]

for i in xrange(0,1000):
    x_GP_1, y_GP_1, sigma_GP_1 = GP_Nd(temp,temp_out_1[:,i])
    x_pred_1.append(x_GP)
    y_pred_1.append(y_GP)
    sigma.append(sigma_GP)
    
    print i



In [ ]:
shape(outputs_1)